# Collection of examples for annotation using the GTF database and gffutils

In [1]:
import pandas as pd
import gffutils
import os
from collections import defaultdict

In [4]:
annotation_file = '/projects/ps-yeolab/genomes/mm10/gencode/m10/gencode.vM10.annotation.gtf'
db_file = '/projects/ps-yeolab3/bay001/annotations/gencode.vM10.annotation.db'

def build_db(annotation_file, db_file, force=True, disable_infer_genes=True, disable_infer_transcripts=True):
    db = gffutils.create_db(
        annotation_file, dbfn=db_file, force=force, # change to True if we need to create a new db
        keep_order=True, merge_strategy='merge', sort_attribute_values=True,
        disable_infer_genes=disable_infer_genes,
        disable_infer_transcripts=disable_infer_transcripts
    )
build_db(annotation_file, db_file)

In [ ]:
db_file = '/projects/ps-yeolab/genomes/mm10/gencode/gencode.vM3.annotation.gtf.db'
# db_file = '/projects/ps-yeolab/genomes/hg19/gencode_v19/gencode.v19.annotation.gtf.db'
DATABASE = gffutils.FeatureDB(db_file)

In [ ]:
def gene_id_to_name(db):
    '''
    Returns a dictionary containing a gene_id:name translation
    Note: may be different if the 'gene_id' or 'gene_name' 
    keys are not in the source GTF file
    (taken from gscripts.region_helpers)
    '''
    genes = db.features_of_type('gene')
    gene_name_dict = {}
    for gene in genes:
        gene_id = gene.attributes['gene_id'][0] if type(gene.attributes['gene_id']) == list else gene.attributes['gene_id']
        try:
            gene_name_dict[gene_id] = gene.attributes['gene_name'][0]
        except KeyError:
            print(gene.attributes.keys())
            print("Warning. Key not found for {}".format(gene))
            return 1
    return gene_name_dict

gene_id_to_name_dictionary = gene_id_to_name(DATABASE)
# gene_id_to_name_dictionary['ENSG00000100320.18']
gene_id_to_name_dictionary['ENSMUSG00000092210.1']

In [ ]:
def gene_name_to_id(db):
    '''
    given a gene name, returns a list of associated Gene IDs (one-to-many)
    '''
    genes = db.features_of_type('gene')
    gene_name_dict = defaultdict(list)
    for gene in genes:
        try:
            gene_name_dict[gene.attributes['gene_name'][0]].append(gene.attributes['gene_id'][0])
        except KeyError as e:
            print("Warning. Key not found for {}".format(gene))
            return 1
    return gene_name_dict

gene_name_to_id_dictionary = gene_name_to_id(DATABASE)
gene_name_to_id_dictionary['RBFOX2']

In [ ]:
def gene_name_to_transcript(db):
    '''
    given a gene name, returns a list of associated transcript IDs (one-to-many)
    '''
    genes = db.features_of_type('transcript')
    gene_name_dict = defaultdict(list)
    for gene in genes:
        try:
            gene_name_dict[gene.attributes['gene_name'][0]].append(gene.attributes['transcript_id'][0])
        except KeyError as e:
            print("Warning. Key not found for {}".format(gene))
            return 1
    return gene_name_dict

gene_name_to_id_dictionary = gene_name_to_transcript(DATABASE)
gene_name_to_id_dictionary['RBFOX2']

In [ ]:
def id_to_exons(db, identifier):
    '''
    takes the gene or transcript id and returns exon positions
    '''
    exons = []
    for i in db.children(identifier, featuretype='exon', order_by='start'):
        exons.append(i)
    return exons

id_to_exons(DATABASE,'ENST00000473487.2')

In [ ]:
def position_to_features(db, chrom, start, end, strand='', completely_within=True):
    '''
    takes a coordinate and returns all the features overlapping 
    (either completely contained or partially overlapping the region).
    '''
    if strand == '+' or strand == '-':
        return list(
            db.region(
                region=(chrom, start, end), strand=strand, completely_within=completely_within
            )
        )
    else:
        return list(
            db.region(
                region=(chrom, start, end), completely_within=completely_within
            )
        )
# get all features corresponding to the genomic coordinates (True if feature must be entirely contained within region)
features = position_to_features(DATABASE,'chr19', 1000000, 1001000, completely_within=True)

# print all gene names associated with these features
print([f.attributes['gene_name'] for f in features])

In [ ]:
# Feature objects embed all information as a dictionary
# See: http://pythonhosted.org/gffutils/attributes.html

for f in features:
    print('{}, {}, {}, {}'.format(
        f.attributes['gene_name'], # list of associated gene names
        f.start, # start of feature
        f.end, # end of feature
        f.attributes['transcript_type'])) # type of feature

# Use some genomic coordinate list to intersect with features
- for each region, return a list of features that are completely contained within that region.
- returns an empty list for regions that do not contain any features

In [ ]:
df = pd.read_table('/projects/ps-yeolab3/bay001/annotations/small_bed.bed3', names=['chrom','start','end'], index_col=0)
df

In [ ]:
features = {}
for ix, row in df.iterrows():
    features[ix] = position_to_features(DATABASE, row['chrom'], row['start'], row['end'], True)

In [ ]:
for name, region_list in features.iteritems():
    for region in region_list:
        print(name, region.featuretype, region.attributes['transcript_type'][0], region.attributes['gene_name'][0])


# Using with pybedtools

In [ ]:
import pybedtools

In [ ]:
interval = pybedtools.create_interval_from_list(['chr1','13000000','13003000','some_interval','0','-'])


In [ ]:
def bedtool_to_features(db, interval, completely_within):
    """
    
    takes a coordinate and returns all the features overlapping 
    (either completely contained or partially overlapping the region).
    
    Parameters
    ----------
    db : sqlite3 database
    interval : pybedtools.Interval
        interval object
    completely_within : bool
        True if the features returned must be completely contained
        within the region. False if the features need only to be
        partially overlapping the region.
        
    Returns
    -------
    region_list: list
        list of Features corresponding to overlapping/contained
        features intersecting a region.
    """
    return position_to_feature(
        db,
        interval.chrom,
        interval.start,
        interval.end,
        interval.strand,
        completely_within
    )

bedtool_to_features(DATABASE, interval, True)

# Working with raw GTF files (no database)

In [ ]:
gtf_file = '/projects/ps-yeolab/genomes/hg19/gencode_v19/gencode.v19.annotation.gtf'
# gtf_file = '/projects/ps-yeolab3/bay001/annotations/c_elegans.PRJNA13758.WS257.canonical_geneset.gtf'
GTF_NAMES = ['chrom','source','feature_type','start','end','.','strand','.','attributes']


In [ ]:

def get_feature_type_set(gtf_file):
    """
    from a GTF file, extract the set of feature_types
    (feature_types is the third column, normally)
    This might be useful for figuring out the priority for annotation.
    
    Parameters
    ----------
    gtf_file

    Returns
    -------

    """
    gtf_df = pd.read_table(
        gtf_file,
        names=GTF_NAMES,
        comment='#'
    )
    return set(gtf_df['feature_type'])


def get_attribute_type_set(gtf_file, attribute_type):
    """
    from a GTF file, extract the set of attribute_types
    (attribute_types is one of those fields contained within the 9th column)
    This might be useful for figuring out the priority for annotation.
    
    Parameters
    ----------
    gtf_file : basestring
    attribute_type : basestring

    Returns
    -------

    """

    gtf_df = pd.read_table(
        gtf_file,
        names=GTF_NAMES,
        comment='#'
    )
    regex_filter = '{} \"([\w\s\d -]+)\"'.format(attribute_type)
    return set(gtf_df['attributes'].str.extract(regex_filter, expand=False))


In [ ]:
# in C elegans GFF
get_feature_type_set(gtf_file)

In [ ]:
# in C elegans GFF
get_attribute_type_set(gtf_file, 'biotype')

In [ ]:
# in Human GENCODE
get_feature_type_set(gtf_file)

In [ ]:
# in Human GENCODE
get_attribute_type_set(gtf_file, 'transcript_type')